**Performing first trials in jupyter for Tyne and Wear**

**1. Variables declarations**

NB: to be edited depending on who runs this

In [96]:
import pandas as pd
import numpy as np
import geopandas as gpd

msoa_full_file = "./../data/MSOAs_TyneWear_full.csv"
msoa_list_file = "./../data/MSOAs_TyneWear.csv"
lad_list_file = "./../data/LADs_TyneWear.csv"


house_prices_lsoa_xls_file = "./../data/hpssadataset46medianpricepaidforresidentialpropertiesbylsoa.xls"

greenspace_file = "./../data/gis/OS Open Greenspace (GPKG) GB/data/opgrsp_gb.gpkg"

LSOA_boundaries_file = "./../data/gis/LSOA_(Dec_2011)_Boundaries_Super_Generalised_Clipped_(BSC)_EW_V3/"
MSOA_boundaries_file = ""

output_folder = "./../output/"

**2. Variables import**

A. Tables import

In [97]:
msoas_full_df = pd.read_csv(msoa_full_file)
msoas_list_df = pd.read_csv(msoa_list_file)
lad_list_df = pd.read_csv(lad_list_file)

houseprice_excel_tab = pd.read_excel(house_prices_lsoa_xls_file,
                                     sheet_name = "Data") # name of excel tab within the spreadsheet which contains our desired data


some data edits

In [173]:
# editing the dataframe (contains empty rows and other un-necessary stuff inherited from excel)
# a. first 4 rows are empty
# b. 5th row contains the columns names

houseprice_df_notitle = houseprice_excel_tab.iloc[4:] # eliminating first 4 rows
houseprice_df_notitle.columns = houseprice_df_notitle.iloc[0] # attributing columns names from current first row
houseprice_df = houseprice_df_notitle.iloc[1:] # eliminating first row (with names)

# note: think to edit this when reading the file in, already,as in
# https://stackoverflow.com/questions/48036296/using-read-excel-with-converters-for-reading-excel-file-into-pandas-dataframe-re

# checking data type for columns with numerical values
houseprice_df.iloc[:, 0:6].dtypes
# we have a total of 113 columns, checking the last 6 ones
houseprice_df.iloc[:,-6: ].dtypes
# let's eliminate the last 3 columns (empty?)
houseprice_df = houseprice_df.iloc[ : , :-3]
houseprice_df.iloc[:,-6: ].dtypes
# numerical values are read in a type 'object', but we can't change this in the reading command above:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
# in fact 'dtype' option changes it for all the columns
# so we do that only after import, here:

4
Year ending Dec 2020    object
Year ending Mar 2021    object
Year ending Jun 2021    object
Year ending Sep 2021    object
Year ending Dec 2021    object
Year ending Mar 2022    object
dtype: object

In [174]:
colnames_houseprice = houseprice_df.columns.to_list()
numerical_colnames = colnames_houseprice[4:] # selecting only names names of the columns that contain numerical values

In [ ]:
# houseprice_df.head()
# # changing type to numeric for columns from position 4 to end
# # houseprice_df.iloc[:,4:].apply(pd.to_numeric, errors='coerce')
# # houseprice_df.iloc[:,4:].astype(float)
# # houseprice_df.iloc[:,4:] = pd.to_numeric(houseprice_df.iloc[:,4:].stack(),
# #                                          errors='coerce').unstack()
# numerical_cols = houseprice_df.columns.str.startswith('Year ending')
# # object_cols = ~houseprice_df.columns.str.startswith('Year ending')
# houseprice_df.loc[:,houseprice_df.columns.str.startswith('Year ending')]

In [175]:
houseprice_df[numerical_colnames] = houseprice_df[numerical_colnames].apply(pd.to_numeric,
                                                                            errors='coerce')
houseprice_df.iloc[:,-6:].dtypes # worked
# https://stackoverflow.com/questions/36814100/pandas-to-numeric-for-multiple-columns

4
Year ending Dec 2020    float64
Year ending Mar 2021    float64
Year ending Jun 2021    float64
Year ending Sep 2021    float64
Year ending Dec 2021    float64
Year ending Mar 2022    float64
dtype: object

selecting region (case: Tyne and Wear county)

In [131]:
# NOTE: the common field between the houseprice file and regional file is the LAD list
region_LADs_codes_list = lad_list_df["LAD20CD"].to_numpy()

# generating list of LSOAs code for the region
#
# generating list of MSOAs code for the region
#

houseprice_lsoa_region = houseprice_df[houseprice_df_num["Local authority code"].isin(region_LADs_codes_list)]
houseprice_lsoa_region.dtypes

KeyError: 'Local authority code'

In [ ]:
# dataframe with LSOAs code and name
lsoas_region_list = houseprice_lsoa_region[["LSOA code", "LSOA name"]]
# creating the list of the region codes for indexing (filtering) purposes
lsoas_region_codes = lsoas_region_list["LSOA code"].to_numpy()

B. Importing geospatial data

In [ ]:
# importing greenspace (gpkg)
# note, available layers are: "GreenspaceSite" (green areas) and "AccessPoint" (actual access points)
greenspace = gpd.read_file(greenspace_file,
                           layer = "GreenspaceSite")
greenspace.head() 

In [ ]:
greenspace.plot()

In [ ]:
accesspoints_gs = gpd.read_file(greenspace_file,
                           layer = "AccessPoint")
accesspoints_gs.head() 

In [ ]:
accesspoints_gs.plot()

In [ ]:
# importing LSOA admin boundaries (shp or json)
lsoas_shp = gpd.read_file(LSOA_boundaries_file)
lsoas_shp.head()

In [ ]:
# selecting LSOAs from Tyne and Wear
lsoas_shp_region = lsoas_shp.query('LSOA11CD in @lsoas_region_codes')

In [ ]:
lsoas_shp_region.plot()

**3. Data analysis**

Associating data to geospatial data (or viceversa, adding geography to data)

In [ ]:
# searching for common field between the 2 tables
houseprice_lsoa_region.columns

In [ ]:
lsoas_shp_region.columns

In [ ]:
# lsoa_gdf_with_houseprice = lsoas_shp_region.merge(houseprice_lsoa_region, 
#                                                   on='')
lsoa_gdf_region_with_houseprice = pd.merge(lsoas_shp_region,
                                 houseprice_lsoa_region,
                                 left_on='LSOA11CD',
                                 right_on="LSOA code")

In [ ]:
lsoa_gdf_region_with_houseprice.columns
N=3
lsoa_gdf_region_with_houseprice.iloc[:, 118:124].dtypes # checking type of column
# ... actually price is an 'object' type ... converting it earlier on

plotting map... trials

In [ ]:
# define legend keys
lgnd_kwds = {'title': 'Median house price per area',
             'loc': 'upper right',
             'bbox_to_anchor': (1.8, 1),
             'ncol':2}

lsoa_gdf_region_with_houseprice.plot(column="Year ending Mar 2022",
                                     cmap="RdYlBu",
                                     categorical=True,
                                     legend=True,
                                     legend_kwds=lgnd_kwds)